In [1]:
#1.import libraries 
import numpy as np
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from keras.layers import Dense, GRU

In [2]:
# 2. Read csv file 
df = pd.read_csv('BID.csv')
df =df.reset_index()['Price']
df

0       11054.8
1       10574.2
2        9973.4
3       10213.7
4       10153.6
         ...   
2466    41500.0
2467    41000.0
2468    41200.0
2469    41300.0
2470    41800.0
Name: Price, Length: 2471, dtype: float64

In [3]:
# 3. Scale data
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df).reshape(-1,1))

In [17]:
# 4.Train 8 Test 2
train_size = int(0.8 * len(df1))
test_size = int(0.2 * len(df1))

train_data = df1[:train_size]
test_data = df1[train_size:train_size+test_size]

In [18]:
# 5.Create Dataset
import numpy
# convert an array of values into a dataset matrix, split data into time column
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0] 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [19]:
#6. Reshape into X=t,t+1,t+2..t+99 and Y=t+100
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [20]:
# 8. Define GRU Model
model = xgb.XGBRegressor(
             colsample_bynode=1, colsample_bytree=1, gamma=0.01,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=8, min_child_weight=1, n_estimators=400,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=42)

In [21]:
# 9. Fit model, train
model.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=1, colsample_bytree=1,
             device=None, early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.01, grow_policy=None,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=0, max_depth=8,
             max_leaves=None, min_child_weight=1, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=400,
             n_jobs=1, nthread=None, num_parallel_tree=None, ...)

In [22]:
# 10. Predcit data 
train_predict=model.predict(X_train)
y_pred=model.predict(X_test)

In [23]:
# 11. y_pred, y_pred_val
train_predict=scaler.inverse_transform(train_predict.reshape(-1,1))
y_pred=scaler.inverse_transform(y_pred.reshape(-1,1))

In [24]:
train_size1 = int(0.8 * len(df))
test_size1 = int(0.2 * len(df))

train_data1 = df[:train_size1]
test_data1 = df[train_size1:train_size1+test_size1]

# re make predictions on the testing set
x_test1 = np.array(test_data1.index).reshape(-1, 1)
y_test1 = np.array(test_data1)

In [51]:
# 11. accurate algorithms
#RMSE
test_rmse = np.sqrt(np.mean((y_pred - y_test1)**2))
print('Testing RMSE:', round(test_rmse,3))

#MAPE
test_mape = np.mean(np.abs((y_test1 - y_pred) / y_test1)) * 100
print('Testing MAPE:', round(test_mape, 3))

#MAE
test_mae = np.mean(np.abs(y_pred - y_test1))
print('Testing MAE:', round(test_mae, 3))

#MLSE
msle = np.mean((np.log1p(y_pred) - np.log1p(y_test1))**2)
print('MSLE:', round(msle, 3))

Testing RMSE: 6222.381
Testing MAPE: 12.569
Testing MAE: 5226.414
MSLE: 0.025


In [52]:
# 13. 
# next 30 days
pred_next = np.array([])

# change data input 
y_predict_to_next = scaler.fit_transform(np.array(y_test1[-time_step:].reshape(1, -1)).reshape(-1, 1))

for i in range(30):
    print(y_predict_to_next)
    y_pred_next = model.predict(y_predict_to_next.reshape(1, -1))
    y_predict_to_next = np.append(y_predict_to_next, y_pred_next[0])[1:]

    pred_next = np.append(pred_next, y_pred_next[0])

# nghich dao
next30days_predict_inverse = pd.DataFrame(scaler.inverse_transform(pred_next.reshape(-1, 1)))


[[0.91219512]
 [1.        ]
 [0.94634146]
 [0.93170732]
 [0.97560976]
 [0.95609756]
 [0.91219512]
 [0.74634146]
 [0.76585366]
 [0.85365854]
 [0.79512195]
 [0.84390244]
 [0.80487805]
 [0.52195122]
 [0.65853659]
 [0.67804878]
 [0.66829268]
 [0.67804878]
 [0.6195122 ]
 [0.63902439]
 [0.67804878]
 [0.71707317]
 [0.7902439 ]
 [0.84390244]
 [0.82439024]
 [0.82926829]
 [0.79512195]
 [0.74634146]
 [0.79512195]
 [0.79512195]
 [0.77073171]
 [0.77560976]
 [0.67317073]
 [0.6195122 ]
 [0.66829268]
 [0.6195122 ]
 [0.70243902]
 [0.6097561 ]
 [0.53658537]
 [0.54146341]
 [0.53658537]
 [0.53658537]
 [0.49756098]
 [0.26341463]
 [0.35609756]
 [0.26829268]
 [0.32682927]
 [0.30243902]
 [0.31219512]
 [0.30731707]
 [0.32195122]
 [0.35609756]
 [0.27317073]
 [0.23902439]
 [0.11219512]
 [0.13170732]
 [0.16097561]
 [0.14146341]
 [0.23902439]
 [0.18536585]
 [0.16585366]
 [0.30731707]
 [0.2097561 ]
 [0.13170732]
 [0.11219512]
 [0.27317073]
 [0.25853659]
 [0.28780488]
 [0.28780488]
 [0.39512195]
 [0.3902439 ]
 [0.32

In [47]:
next30days_predict_inverse

0
0   41294.288969
1   41262.240338
2   41322.247612
3   41409.924856
4   41386.350220
5   41409.924856
6   41420.414521
7   41443.541026
8   41548.218346
9   41548.218346
10  41548.218346
11  41531.861140
12  41531.861140
13  41531.861140
14  41531.861140
15  41531.861140
16  41541.587120
17  41541.587120
18  41541.587120
19  41557.944326
20  41541.587120
21  41532.801388
22  41523.075408
23  41523.075408
24  41532.801388
25  41516.444182
26  41516.444182
27  41516.444182
28  41516.444182
29  41541.587120

In [48]:
# 14.plots
plt.figure(figsize =(16, 9))
train_data_index = pd.RangeIndex(start=0, stop=train_size, step=1)
plt.plot(train_data_index,train_data1)
test_data_index = pd.RangeIndex(start=train_size, stop=train_size+test_size, step=1)
plt.plot(test_data_index,test_data1)
test_data_index = pd.RangeIndex(start=train_size+101, stop=train_size+test_size, step=1)
plt.plot(test_data_index,(y_pred))
prediect_data_index = pd.RangeIndex(start=len(df1)-1, stop=len(df1)+29, step=1)
plt.plot(prediect_data_index,next30days_predict_inverse)
plt.legend(['Train','Test','Predict','Predict30days'])
plt.show()

In [49]:
train_data


array([[0.07418187],
       [0.06248662],
       [0.04786633],
       ...,
       [0.65688088],
       [0.66418128],
       [0.66174781]])